In [ ]:
!pip install requests

import requests
import json

ES_URL        = "http://localhost:9200"
ES_USER       = "elastic"
ES_PASS       = "changeme"
ENDPOINT_NAME = "elser-local"

auth = (ES_USER, ES_PASS)
json_hdr = {"Content-Type": "application/json"}

# 0) sanity check
r = requests.get(f"{ES_URL}/_cluster/health", auth=auth, timeout=30)
r.raise_for_status()

# 1) delete existing endpoint if exists
requests.delete(f"{ES_URL}/_inference/{ENDPOINT_NAME}", auth=auth, timeout=30)

# 2) recreate endpoint
create_body = {
    "service": "elser",
    "service_settings": {
        "num_allocations": 1,
        "num_threads": 1
    }
}
r = requests.put(
    f"{ES_URL}/_inference/sparse_embedding/{ENDPOINT_NAME}",
    auth=auth, headers=json_hdr, json=create_body, timeout=60
)
r.raise_for_status()

# 3) tiny infer
infer_body = {"input": "hello world from addis ababa"}
r = requests.post(
    f"{ES_URL}/_inference/sparse_embedding/{ENDPOINT_NAME}",
    auth=auth, headers=json_hdr, json=infer_body, timeout=60
)
r.raise_for_status()

print(json.dumps(r.json(), indent=2))
print("\n✅ ELSER endpoint ready & responding.")